<a href="https://colab.research.google.com/github/Offliners/writeup/blob/main/HW7/homework7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [13]:
# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Sat May 15 18:08:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    31W /  70W |   1508MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
# Download link 1
!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

!unzip -o hw7_data.zip

Downloading...
From: https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1
To: /content/hw7_data.zip
7.71MB [00:00, 67.8MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          


## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [15]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

## Import Packages

In [16]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [17]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

## Load Model and Tokenizer




 

In [18]:
# model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  
tokenizer = AutoTokenizer.from_pretrained("nyust-eb210/braslab-bert-drcd-384")

model = AutoModelForQuestionAnswering.from_pretrained("nyust-eb210/braslab-bert-drcd-384")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [19]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [20]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors


## Dataset and Dataloader

In [21]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 100
        self.max_paragraph_len = 384
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 128

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token)
            paragraph_start = max(0, min(mid - self.max_paragraph_len, len(tokenized_paragraph) - self.max_paragraph_len))
            # paragraph_start = 0
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start

            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 10

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [22]:
def evaluate(data, output):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

## Training

In [23]:
num_epoch = 3
validation = True
logging_step = 100
learning_rate = 3e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * num_epoch
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        step += 1

        ##### TODO: Apply linear learning rate decay #####
        scheduler.step()

        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 1 | Step 100 | loss = 1.522, acc = 0.460
Epoch 1 | Step 200 | loss = 0.860, acc = 0.644
Epoch 1 | Step 300 | loss = 0.829, acc = 0.690
Epoch 1 | Step 400 | loss = 0.792, acc = 0.695
Epoch 1 | Step 500 | loss = 0.645, acc = 0.719
Epoch 1 | Step 600 | loss = 0.639, acc = 0.717
Epoch 1 | Step 700 | loss = 0.679, acc = 0.712
Epoch 1 | Step 800 | loss = 0.636, acc = 0.735
Epoch 1 | Step 900 | loss = 0.630, acc = 0.722
Epoch 1 | Step 1000 | loss = 0.624, acc = 0.726
Epoch 1 | Step 1100 | loss = 0.631, acc = 0.753
Epoch 1 | Step 1200 | loss = 0.579, acc = 0.757
Epoch 1 | Step 1300 | loss = 0.597, acc = 0.752
Epoch 1 | Step 1400 | loss = 0.532, acc = 0.787
Epoch 1 | Step 1500 | loss = 0.513, acc = 0.770
Epoch 1 | Step 1600 | loss = 0.542, acc = 0.776
Epoch 1 | Step 1700 | loss = 0.614, acc = 0.765
Epoch 1 | Step 1800 | loss = 0.539, acc = 0.782
Epoch 1 | Step 1900 | loss = 0.496, acc = 0.806
Epoch 1 | Step 2000 | loss = 0.501, acc = 0.774
Epoch 1 | Step 2100 | loss = 0.532, acc = 0.776
E


Validation | Epoch 1 | acc = 0.780


Epoch 2 | Step 100 | loss = 0.256, acc = 0.869
Epoch 2 | Step 200 | loss = 0.283, acc = 0.867
Epoch 2 | Step 300 | loss = 0.268, acc = 0.861
Epoch 2 | Step 400 | loss = 0.268, acc = 0.881
Epoch 2 | Step 500 | loss = 0.234, acc = 0.887
Epoch 2 | Step 600 | loss = 0.251, acc = 0.880
Epoch 2 | Step 700 | loss = 0.248, acc = 0.900
Epoch 2 | Step 800 | loss = 0.307, acc = 0.867
Epoch 2 | Step 900 | loss = 0.233, acc = 0.887
Epoch 2 | Step 1000 | loss = 0.247, acc = 0.885
Epoch 2 | Step 1100 | loss = 0.235, acc = 0.883
Epoch 2 | Step 1200 | loss = 0.285, acc = 0.867
Epoch 2 | Step 1300 | loss = 0.245, acc = 0.885
Epoch 2 | Step 1400 | loss = 0.266, acc = 0.880
Epoch 2 | Step 1500 | loss = 0.275, acc = 0.875
Epoch 2 | Step 1600 | loss = 0.237, acc = 0.869
Epoch 2 | Step 1700 | loss = 0.246, acc = 0.890
Epoch 2 | Step 1800 | loss = 0.235, acc = 0.890
Epoch 2 | Step 1900 | loss = 0.266, acc = 0.868
Epoch 2 | Step 2000 | loss = 0.230, acc = 0.893
Epoch 2 | Step 2100 | loss = 0.252, acc = 0.876
E


Validation | Epoch 2 | acc = 0.787


Epoch 3 | Step 100 | loss = 0.095, acc = 0.939
Epoch 3 | Step 200 | loss = 0.092, acc = 0.957
Epoch 3 | Step 300 | loss = 0.108, acc = 0.942
Epoch 3 | Step 400 | loss = 0.105, acc = 0.955
Epoch 3 | Step 500 | loss = 0.095, acc = 0.951
Epoch 3 | Step 600 | loss = 0.096, acc = 0.956
Epoch 3 | Step 700 | loss = 0.106, acc = 0.946
Epoch 3 | Step 800 | loss = 0.108, acc = 0.942
Epoch 3 | Step 900 | loss = 0.098, acc = 0.947
Epoch 3 | Step 1000 | loss = 0.097, acc = 0.945
Epoch 3 | Step 1100 | loss = 0.104, acc = 0.951
Epoch 3 | Step 1200 | loss = 0.088, acc = 0.942
Epoch 3 | Step 1300 | loss = 0.138, acc = 0.925
Epoch 3 | Step 1400 | loss = 0.103, acc = 0.940
Epoch 3 | Step 1500 | loss = 0.085, acc = 0.952
Epoch 3 | Step 1600 | loss = 0.103, acc = 0.949
Epoch 3 | Step 1700 | loss = 0.112, acc = 0.946
Epoch 3 | Step 1800 | loss = 0.102, acc = 0.939
Epoch 3 | Step 1900 | loss = 0.114, acc = 0.944
Epoch 3 | Step 2000 | loss = 0.085, acc = 0.955
Epoch 3 | Step 2100 | loss = 0.098, acc = 0.951
E


Validation | Epoch 3 | acc = 0.794
Saving Model ...


## Testing

In [28]:
# Postprocessing

def postprocessing(result, index):
    if result:
        if result.find('[UNK]') != -1:
            print(f'ID {index} Before : {result}')
            if result[:result.find('[UNK]')] != '' and result[result.find('[UNK]') + len('[UNK]') + 1:] != '':
                start = test_paragraphs[test_questions[index]['paragraph_id']].find(result[:result.find('[UNK]')]) + len(result[: result.find('[UNK]')])
                end = test_paragraphs[test_questions[index]['paragraph_id']].find(result[result.find('[UNK]') + len('[UNK]') + 1:])
                target = test_paragraphs[test_questions[index]['paragraph_id']][start: end - 1]

                if len(target) < 10:
                    print(f'Target: {target}')
                    result = result.replace('[UNK]', target)
                else:
                    start = test_paragraphs[test_questions[index]['paragraph_id']].find(result[result.find('[UNK]') - 1])
                    target = test_paragraphs[test_questions[index]['paragraph_id']][start + 1]
                    result = result.replace('[UNK]', target)
                print(f'ID {index} After  : {result}')
            else:
                start = test_paragraphs[test_questions[index]['paragraph_id']].find(result[result.find('[UNK]') - 1])
                target = test_paragraphs[test_questions[index]['paragraph_id']][start + 1]
                result = result.replace('[UNK]', target)
                print(f'ID {index} After  : {result}')
             
        if result[0] == '，' or result[0] == '。':
            print(f'ID {index} Before : {result}')
            result = result[1:]
            print(f'ID {index} After  : {result}')

        if result[0] == '《' and result[-1] != '》' and result.find('》') == -1:
            print(f'ID {index} Before : {result}')
            result = result + '》'
            print(f'ID {index} After  : {result}')
        
        if result[0] != '《' and result[-1] == '》' and result.find('《') == -1:
            print(f'ID {index} Before : {result}')
            result = '《' + result  
            print(f'ID {index} After  : {result}')
        
    return result

In [29]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
                result[i] = postprocessing(result[i], i)
                f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...



ID 49 Before : 自大型購物中心[UNK]開幕
Target: MegaBox
ID 49 After  : 自大型購物中心MegaBox開幕
ID 250 Before : 溥[UNK]
ID 250 After  : 溥儁
ID 332 Before : 沒有觀察到任何語言純[UNK]以力道來區分不同輔音
Target: 綷
ID 332 After  : 沒有觀察到任何語言純綷以力道來區分不同輔音
ID 340 Before : [UNK]人國
ID 340 After  : 志人國
ID 563 Before : 馬[UNK]
ID 563 After  : 馬馼
ID 635 Before : 東晉常[UNK]
ID 635 After  : 東晉常璩
ID 938 Before : [UNK]稻
ID 938 After  : 米稻
ID 991 Before : 白[UNK]紀滅絕事件
ID 991 After  : 白堊紀滅絕事件
ID 1212 Before : [UNK]州縣實際解散
ID 1212 After  : ，州縣實際解散
ID 1212 Before : ，州縣實際解散
ID 1212 After  : 州縣實際解散
ID 1423 Before : 抗佝[UNK]病維他命
Target: 僂
ID 1423 After  : 抗佝僂病維他命
ID 1494 Before : [UNK]以下的聲音對基底膜的影響
ID 1494 After  : 。以下的聲音對基底膜的影響
ID 1494 Before : 。以下的聲音對基底膜的影響
ID 1494 After  : 以下的聲音對基底膜的影響
ID 1496 Before : 外星人入侵地球電影，例如1996年的[UNK]星際終結者，由於生命科學的突飛猛進，例如1993年的侏羅紀公園與1997年的千鈞一髮。這十年當中，電影對於特效的重要性扮演不可言喻的角色，例如魔鬼終結者2，侏羅紀公園都有大量的電腦動畫。隨著軟體的複雜性上升，便於製作更複雜的效果，也使製片人可以利用電腦提高動畫的品質，例如1995年的攻殼機動隊與1999年的鐵巨人。在21世紀的這10年中，科幻電影的最大特徵就是超級英雄電影大量的出現，例如：《[UNK]戰警》、《蜘蛛人》、《綠巨人浩克》等等都是在這十年開拍

In [30]:
from google.colab import files
files.download("result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Reference**
Source: Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW07/HW07.ipynb)